# Hive - Bucketing

In [ ]:
1.'Hive Bucketing' - is a technique to split the data into more manageable files, (By specifying the number of buckets to create). The value of the bucketing column will be hashed by a user-defined number into buckets.

2.Bucketing can be created on just 'one column', you can also create bucketing on a partitioned table to further split 
  the data to improve the query performance of the partitioned table.

3.Each bucket is stored as a file within the table’s directory or the partitions directories on HDFS.

4.Records with the same value in a column will always be stored in the same bucket.

5.Hive bucketing commonly created in two scenarios.
  i. Create a bucket on top of the Partitioned table to further divide the table for better query performance.
  ii.Create Bucketing on the table where you cannot choose the partition column due to (too many distinct values on columns).

# Hive Create Bucketing Table

In [ ]:
CREATE TABLE zipcodes(
RecordNumber int,
Country string,
City string,
Zipcode int)
PARTITIONED BY(state string)
CLUSTERED BY (Zipcode) INTO 32 BUCKETS #syntax for bucketing Creation
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ',';

#Load Data into Bucket

#This property is not needed if you are using Hive 2.x or later
set hive.enforce.bucketing = true;

LOAD DATA INPATH '/data/zipcodes.csv' INTO TABLE zipcodes;



# Select Data From Bucket

In [ ]:
Since our zipcode is partitioned on state and bucketing on zipcode, if you use these columns on where condition your query 
returns faster results.

'SELECT * FROM zipcodes WHERE state='PR' and zipcode=704;'
+------------------------+-------------------+----------------------+-------------------+-----------------+s
| zipcodes.recordnumber  | zipcodes.country  |    zipcodes.city     | zipcodes.zipcode  | zipcodes.state  |
+------------------------+-------------------+----------------------+-------------------+-----------------+
| 3                      | US                | SECT LANAUSSE        | 704               | PR              |
| 2                      | US                | PASEO COSTA DEL SUR  | 704               | PR              |
| 4                      | US                | URB EUGENE RICE      | 704               | PR              |
| 1                      | US                | PARC PARQUE          | 704               | PR              |
+------------------------+-------------------+----------------------+-------------------+-----------------+



# How Hive Distribute the Rows Across the Buckets?

In [ ]:
In general, the bucket number is determined by the expression 'hash_function(bucketing_column) mod num_buckets'

Example --
if user_id were an int, and there were 10 buckets, we would expect all user_id’s that end in 0 to be in bucket 1, 
all user_id’s that end in a 1 to be in bucket 2,
all user_id’s that end in a 2 to be in bucket 3, etc

# How to Decide the Number of Buckets?

In [ ]:
Lets take a scenario Where table size is: 2300 MB, HDFS Block Size: 128 MB

Now, Divide 2300/128=17.96

Now, remember number of bucket will always be in the power of 2.

So we need to find n such that 2^n > 17.96

n=5

So, I am going to use number of buckets as 2^5=32.

# Differences Between Hive Partitioning vs Bucketing

In [ ]:
'PARTITIONING                                        	BUCKETING'

Directory is created on HDFS for each partition.    	File is created on HDFS for each bucket.
You can have one or more Partition columns          	You can have only one Bucketing column
You can’t manage the number of partitions to create  	You can manage the number of buckets to create by specifying the count
NA                                                   	Bucketing can be created on a partitioned table
Uses PARTITIONED BY                                 	Uses CLUSTERED BY